In [1]:
# Set filetype environment variable here, before anything is loaded... "LSM" is LSM, "CV" is default and for CV7000 or CV8000
import os
os.environ["FILE_TYPE"] = "LSM"

In [2]:
from models.swarm_job import SwarmJob, RunStrategy
from pathlib import Path
from generate_all_maximum_projections import GenerateAllMaximumProjectionsJob
from generate_all_nuclear_segmentations import GenerateAllNuclearSegmentationsJob
from generate_all_nuclear_masks import GenerateAllNuclearMasksJob
from generate_all_cropped_cell_images import GenerateAllCroppedCellImagesJob
from generate_all_distance_transforms import GenerateAllDistanceTransformsJob
from generate_all_cell_result_lines import GenerateAllCellResultLinesJob
from generate_cell_results_file import GenerateCellResultsFileJob

In [3]:
# Set strategy here -- "RunStrategy.SWARM" is running on Biowulf/with swarm in parallel, "RunStrategy.LOCAL" is testing a single well/image without parallelization
SwarmJob.run_strategy = RunStrategy.LOCAL

In [4]:
# Set your file system variables here: where are your images and what folders would you like to output to?

source_images = "/data/finne/20210723_Josh/Test/"
results_directory = Path("/data/finne/20210723_Josh/Results")
MIP_directory = results_directory / "MIPs_and_z_centers/"
nuclear_segmentations_directory = results_directory / "nuclear_segmentations/" 
nuclear_masks_directory = results_directory / "nuclear_masks/"
distance_transforms_directory = results_directory / "distance_transforms/"
cell_crops_directory = results_directory / "cell_crops/"
cell_result_lines_directory = results_directory / "cell_result_lines/"

log_files_directory = results_directory/ "logs/"

# CellPose takes as an argument the diameter of the cells. Looking at some images and rounding up works fairly well to start.
# If you're not sure and want to test a few options, first run the maximum projection script and then use the test_nuclear_segmentation.ipynb notebook.

DAPI_channel = 3
diameter = 100

In [ ]:
# Maximum projections!
GenerateAllMaximumProjectionsJob(source_images, 
                                 MIP_directory, 
                                 log_files_directory).run()

In [ ]:
# Nuclear segmentations!
GenerateAllNuclearSegmentationsJob(MIP_directory, 
                                   nuclear_segmentations_directory, 
                                   log_files_directory, 
                                   diameter, DAPI_channel).run()

In [ ]:
# Crop nuclear segmentations for individual nuclear masks!
GenerateAllNuclearMasksJob(nuclear_segmentations_directory, 
                           nuclear_masks_directory, 
                           log_files_directory).run()

In [ ]:
# Calculate distance transform for radial position!
GenerateAllDistanceTransformsJob(nuclear_masks_directory, 
                                 distance_transforms_directory, 
                                 log_files_directory).run()

In [ ]:
# Crop cell images using nuclear masks!
GenerateAllCroppedCellImagesJob(MIP_directory, 
                                nuclear_masks_directory, 
                                cell_crops_directory, 
                                log_files_directory, DAPI_channel).run()

In [ ]:
# Cell-level result line with summed intensities!
GenerateAllCellResultLinesJob(MIP_directory, 
                                nuclear_masks_directory, 
                                cell_result_lines_directory, 
                                log_files_directory).run()

In [ ]:
# Find and merge all cell intensities into a single file!
GenerateCellResultsFileJob(cell_result_lines_directory, results_directory).run()